# House pricing Regression Project

In [35]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import plotly.graph_objects as go
import plotly.express as px
import scipy.stats as stats
from IPython.display import display, HTML
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


# plt.style.use('ggplot')
pd.set_option("display.max_columns", None)

### Reading the data frame

In [36]:
FILEPATH = os.path.join(os.getcwd(), "train.csv")
df = pd.read_csv(FILEPATH)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Make some EDA and Visualization 

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

### Summary statistics for numerical features

In [38]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


### Summary statistics for categorical features

In [39]:
df.describe(include="object")

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1452,1460,1460,1460,1423,1423,1422,1423,1422,1460,1460,1460,1459,1460,1460,770,1379,1379,1379,1379,1460,7,281,54,1460,1460
unique,5,2,2,4,4,2,5,3,25,9,8,5,8,6,8,15,16,4,4,5,6,4,4,4,6,6,6,5,2,5,4,7,5,6,3,5,5,3,3,4,4,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,PConc,TA,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal
freq,1151,1454,50,925,1311,1459,1052,1382,225,1260,1445,1220,726,1141,1434,515,504,864,906,1282,647,649,1311,953,430,1256,1428,741,1365,1334,735,1360,380,870,605,1311,1326,1340,3,157,49,1267,1198


### checking NAN values

In [40]:
for column in df.columns:
    print(f"{column}: {df[column].isnull().sum()}")
# df.isnull().sum()

Id: 0
MSSubClass: 0
MSZoning: 0
LotFrontage: 259
LotArea: 0
Street: 0
Alley: 1369
LotShape: 0
LandContour: 0
Utilities: 0
LotConfig: 0
LandSlope: 0
Neighborhood: 0
Condition1: 0
Condition2: 0
BldgType: 0
HouseStyle: 0
OverallQual: 0
OverallCond: 0
YearBuilt: 0
YearRemodAdd: 0
RoofStyle: 0
RoofMatl: 0
Exterior1st: 0
Exterior2nd: 0
MasVnrType: 8
MasVnrArea: 8
ExterQual: 0
ExterCond: 0
Foundation: 0
BsmtQual: 37
BsmtCond: 37
BsmtExposure: 38
BsmtFinType1: 37
BsmtFinSF1: 0
BsmtFinType2: 38
BsmtFinSF2: 0
BsmtUnfSF: 0
TotalBsmtSF: 0
Heating: 0
HeatingQC: 0
CentralAir: 0
Electrical: 1
1stFlrSF: 0
2ndFlrSF: 0
LowQualFinSF: 0
GrLivArea: 0
BsmtFullBath: 0
BsmtHalfBath: 0
FullBath: 0
HalfBath: 0
BedroomAbvGr: 0
KitchenAbvGr: 0
KitchenQual: 0
TotRmsAbvGrd: 0
Functional: 0
Fireplaces: 0
FireplaceQu: 690
GarageType: 81
GarageYrBlt: 81
GarageFinish: 81
GarageCars: 0
GarageArea: 0
GarageQual: 81
GarageCond: 81
PavedDrive: 0
WoodDeckSF: 0
OpenPorchSF: 0
EnclosedPorch: 0
3SsnPorch: 0
ScreenPorch: 0
Pool

### Dropping some unuseful features

In [41]:
df.drop(["Alley", "FireplaceQu", "PoolQC", "Fence", "MiscFeature", "Id", "Neighborhood", "Exterior1st", "Exterior2nd",
        "Condition1", "Condition2", "LotConfig", "BldgType", "MasVnrType"], axis=1, inplace=True)
df.shape

(1460, 67)

In [42]:
def nans(df):
    navalues = 0
    for i in range(len(df.columns)):
        navalues += df.isnull().sum()[i]

    print(navalues)


In [43]:
df.head(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LandSlope,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Gtl,2Story,7,5,2003,2003,Gable,CompShg,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,Gtl,1Story,6,8,1976,1976,Gable,CompShg,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Gtl,2Story,7,5,2001,2002,Gable,CompShg,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Gtl,2Story,7,5,1915,1970,Gable,CompShg,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,Gtl,2Story,8,5,2000,2000,Gable,CompShg,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000
5,50,RL,85.0,14115,Pave,IR1,Lvl,AllPub,Gtl,1.5Fin,5,5,1993,1995,Gable,CompShg,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,700,10,2009,WD,Normal,143000
6,20,RL,75.0,10084,Pave,Reg,Lvl,AllPub,Gtl,1Story,8,5,2004,2005,Gable,CompShg,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,0,8,2007,WD,Normal,307000
7,60,RL,NaN,10382,Pave,IR1,Lvl,AllPub,Gtl,2Story,7,6,1973,1973,Gable,CompShg,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,350,11,2009,WD,Normal,200000
8,50,RM,51.0,6120,Pave,Reg,Lvl,AllPub,Gtl,1.5Fin,7,5,1931,1950,Gable,CompShg,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,0,4,2008,WD,Abnorml,129900
9,190,RL,50.0,7420,Pave,Reg,Lvl,AllPub,Gtl,1.5Unf,5,6,1939,1950,Gable,CompShg,0.0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,GasA,Ex,Y,SBrkr,1077,0,0,1077,1,0,1,0,2,2,TA,5,Typ,2,Attchd,1939.0,RFn,1,205,Gd,TA,Y,0,4,0,0,0,0,0,1,2008,WD,Normal,118000


### Apply some feature engineering

In [44]:
# adding more features
df["total_rooms"] = df["FullBath"] + df["HalfBath"] + df["BedroomAbvGr"] 
df["FlrSF"] = df["1stFlrSF"] + df["2ndFlrSF"] + df["LowQualFinSF"]
# df["LowQualFinSF"].unique()

### Calculate the correlation matrix

In [45]:
# Calculate the correlation matrix
correlation_matrix = df.corr(method='spearman', numeric_only=True)

# Assuming 'SalePrice' is the target variable
target_correlation = correlation_matrix['SalePrice'].sort_values(ascending=False)

target_correlation

SalePrice        1.000000
OverallQual      0.809829
FlrSF            0.731310
GrLivArea        0.731310
GarageCars       0.690711
YearBuilt        0.652682
GarageArea       0.649379
FullBath         0.635957
TotalBsmtSF      0.602725
GarageYrBlt      0.593788
1stFlrSF         0.575408
YearRemodAdd     0.571159
TotRmsAbvGrd     0.532586
Fireplaces       0.519247
total_rooms      0.514494
OpenPorchSF      0.477561
LotArea          0.456461
MasVnrArea       0.421309
LotFrontage      0.409076
WoodDeckSF       0.353802
HalfBath         0.343008
BsmtFinSF1       0.301871
2ndFlrSF         0.293598
BedroomAbvGr     0.234907
BsmtFullBath     0.225125
BsmtUnfSF        0.185197
ScreenPorch      0.100070
MoSold           0.069432
3SsnPorch        0.065440
PoolArea         0.058453
MSSubClass       0.007192
BsmtHalfBath    -0.012189
YrSold          -0.029899
BsmtFinSF2      -0.038806
MiscVal         -0.062727
LowQualFinSF    -0.067719
OverallCond     -0.129325
KitchenAbvGr    -0.164826
EnclosedPorc

### Select features with Low correlation to saleprice

In [46]:
# Set the correlation threshold
threshold = 0.5

# Select features with Low correlation with the target variable
low_correlation_features = target_correlation[abs(target_correlation) < threshold].index

print(low_correlation_features)

Index(['OpenPorchSF', 'LotArea', 'MasVnrArea', 'LotFrontage', 'WoodDeckSF',
       'HalfBath', 'BsmtFinSF1', '2ndFlrSF', 'BedroomAbvGr', 'BsmtFullBath',
       'BsmtUnfSF', 'ScreenPorch', 'MoSold', '3SsnPorch', 'PoolArea',
       'MSSubClass', 'BsmtHalfBath', 'YrSold', 'BsmtFinSF2', 'MiscVal',
       'LowQualFinSF', 'OverallCond', 'KitchenAbvGr', 'EnclosedPorch'],
      dtype='object')


### Dropping values with least correlation to saleprice

In [47]:
df_dropped = df.drop(low_correlation_features.tolist(), axis=1)
df_dropped.columns

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LandSlope', 'HouseStyle', 'OverallQual', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', 'GrLivArea', 'FullBath', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'SaleType', 'SaleCondition', 'SalePrice', 'total_rooms', 'FlrSF'],
      dtype='object')

In [48]:
df_dropped.describe(include="object")

,MSZoning,Street,LotShape,LandContour,Utilities,LandSlope,HouseStyle,RoofStyle,RoofMatl,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
count,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1423,1423,1422,1423,1422,1460,1460,1460,1459,1460,1460,1379,1379,1379,1379,1460,1460,1460
unique,5,2,4,4,2,3,8,6,8,4,5,6,4,4,4,6,6,6,5,2,5,4,7,6,3,5,5,3,9,6
top,RL,Pave,Reg,Lvl,AllPub,Gtl,1Story,Gable,CompShg,TA,TA,PConc,TA,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,TA,Typ,Attchd,Unf,TA,TA,Y,WD,Normal
freq,1151,1454,925,1311,1459,1382,726,1141,1434,906,1282,647,649,1311,953,430,1256,1428,741,1365,1334,735,1360,870,605,1311,1326,1340,1267,1198


## Work with Ordinal Features

### Making mapping dictionaries

In [49]:
rates_dict = {
    'Ex':6, 
    'Gd':5, 
    'TA':4, 
    'Fa':3, 
    'Po':2, 
    np.nan:1
}
BsmtExposure_dict = {
    'Gd':4, 
    'Av':3, 
    'Mn':2, 
    'No':1, 
    np.nan:0
}
BsmtFinType1_dict = {
    'GLQ':6, 
    'ALQ':5, 
    'BLQ':4, 
    'Rec':3, 
    "LwQ":2,
    "Unf":1,
    np.nan:0
} 
GarageType_dict = {
    '2Types':6, 
    'Attchd':5, 
    'Basment':4, 
    'BuiltIn':3, 
    "CarPort":2,
    "Detchd":1,
    np.nan:0
}
GarageFinish_dict = {
    'Fin':3, 
    'RFn':2, 
    'Unf':1, 
    np.nan:0
}

### Apply mapping dictionaries

In [50]:
# df_dropped["Utilities"] = df_dropped["Utilities"].map(Utilities_dict)
# df_dropped["LandContour"] = df_dropped["LandContour"].map(LandContour_dict)
# df_dropped["LotShape"] = df_dropped["LotShape"].map(LotShape_dict)
# df_dropped["BsmtExposure"] = df_dropped["BsmtExposure"].map(BsmtExposure_dict)
df_dropped["ExterQual"] = df_dropped["ExterQual"].map(rates_dict)
df_dropped["ExterCond"] = df_dropped["ExterCond"].map(rates_dict)
df_dropped["BsmtQual"] = df_dropped["BsmtQual"].map(rates_dict)
df_dropped["BsmtCond"] = df_dropped["BsmtCond"].map(rates_dict)
# df_dropped["BsmtFinType1"] = df_dropped["BsmtFinType1"].map(BsmtFinType1_dict)
# df_dropped["BsmtFinType2"] = df_dropped["BsmtFinType2"].map(BsmtFinType1_dict)
df_dropped["HeatingQC"] = df_dropped["HeatingQC"].map(rates_dict)
df_dropped["KitchenQual"] = df_dropped["KitchenQual"].map(rates_dict)
# df_dropped["GarageType"] = df_dropped["GarageType"].map(GarageType_dict)
# df_dropped["GarageFinish"] = df_dropped["GarageFinish"].map(GarageFinish_dict)
df_dropped["GarageQual"] = df_dropped["GarageQual"].map(rates_dict)
df_dropped["GarageCond"] = df_dropped["GarageCond"].map(rates_dict)
# df_dropped["PavedDrive"] = df_dropped["PavedDrive"].map(PavedDrive_dict)
# df_dropped["SaleType"] = df_dropped["SaleType"].map(SaleType_dict)
# df_dropped["SaleCondition"] = df_dropped["SaleCondition"].map(SaleCondition_dict)


df_dropped.head()


,MSZoning,Street,LotShape,LandContour,Utilities,LandSlope,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,GrLivArea,FullBath,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition,SalePrice,total_rooms,FlrSF
0,RL,Pave,Reg,Lvl,AllPub,Gtl,2Story,7,2003,2003,Gable,CompShg,5,4,PConc,5,4,No,GLQ,Unf,856,GasA,6,Y,SBrkr,856,1710,2,5,8,Typ,0,Attchd,2003.0,RFn,2,548,4,4,Y,WD,Normal,208500,6,1710
1,RL,Pave,Reg,Lvl,AllPub,Gtl,1Story,6,1976,1976,Gable,CompShg,4,4,CBlock,5,4,Gd,ALQ,Unf,1262,GasA,6,Y,SBrkr,1262,1262,2,4,6,Typ,1,Attchd,1976.0,RFn,2,460,4,4,Y,WD,Normal,181500,5,1262
2,RL,Pave,IR1,Lvl,AllPub,Gtl,2Story,7,2001,2002,Gable,CompShg,5,4,PConc,5,4,Mn,GLQ,Unf,920,GasA,6,Y,SBrkr,920,1786,2,5,6,Typ,1,Attchd,2001.0,RFn,2,608,4,4,Y,WD,Normal,223500,6,1786
3,RL,Pave,IR1,Lvl,AllPub,Gtl,2Story,7,1915,1970,Gable,CompShg,4,4,BrkTil,4,5,No,ALQ,Unf,756,GasA,5,Y,SBrkr,961,1717,1,5,7,Typ,1,Detchd,1998.0,Unf,3,642,4,4,Y,WD,Abnorml,140000,4,1717
4,RL,Pave,IR1,Lvl,AllPub,Gtl,2Story,8,2000,2000,Gable,CompShg,5,4,PConc,5,4,Av,GLQ,Unf,1145,GasA,6,Y,SBrkr,1145,2198,2,5,9,Typ,1,Attchd,2000.0,RFn,3,836,4,4,Y,WD,Normal,250000,7,2198


In [51]:
nans(df_dropped)
df_dropped["Electrical"].isna().sum()
df_dropped["Electrical"] = df_dropped["Electrical"].fillna(df_dropped["Electrical"].mode()[0])
df_dropped["GarageYrBlt"] = df_dropped["GarageYrBlt"].fillna(df_dropped["GarageYrBlt"].mode()[0])

357


In [52]:
df_dropped.describe(include="object")

,MSZoning,Street,LotShape,LandContour,Utilities,LandSlope,HouseStyle,RoofStyle,RoofMatl,Foundation,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,CentralAir,Electrical,Functional,GarageType,GarageFinish,PavedDrive,SaleType,SaleCondition
count,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1422,1423,1422,1460,1460,1460,1460,1379,1379,1460,1460,1460
unique,5,2,4,4,2,3,8,6,8,6,4,6,6,6,2,5,7,6,3,3,9,6
top,RL,Pave,Reg,Lvl,AllPub,Gtl,1Story,Gable,CompShg,PConc,No,Unf,Unf,GasA,Y,SBrkr,Typ,Attchd,Unf,Y,WD,Normal
freq,1151,1454,925,1311,1459,1382,726,1141,1434,647,953,430,1256,1428,1365,1335,1360,870,605,1340,1267,1198


### Work with nominal features

In [53]:
cat_columns = []
for col in df_dropped.columns:
    if isinstance(df_dropped[col][0], str):
        cat_columns.append(col)
len(cat_columns)

22

In [54]:
print(df_dropped.shape)
df_dropped = pd.get_dummies(df_dropped, columns=cat_columns, drop_first=True)
# df_dropped.head()
df_dropped.shape

(1460, 45)


(1460, 112)

### splitting the data

In [55]:
X = df_dropped.drop("SalePrice", axis=1)
Y = df_dropped["SalePrice"]
X.shape, Y.shape

((1460, 111), (1460,))

In [56]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1168, 111), (1168,), (292, 111), (292,))

In [57]:
nans(df_dropped)

0


## Models

* `Linear Regression`

In [58]:
# Linear Regression using OLS
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

# Predict
y_pred_train = lin_reg.predict(x_train)
y_pred_test = lin_reg.predict(x_test)

# Evaluate
mse_train_lin = mean_squared_error(y_train, y_pred_train)
mse_test_lin = mean_squared_error(y_test, y_pred_test)

mae_train_lin = mean_absolute_error(y_train, y_pred_train)
mae_test_lin = mean_absolute_error(y_test, y_pred_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"MSE for training using Linear Regression = {mse_train_lin}")
print(f"MSE for testing using Linear Regression = {mse_train_lin}")
print()
print(f"MAE for training using Linear Regression = {mae_train_lin}")
print(f"MAE for testing using Linear Regression = {mae_train_lin}")
print()
print("Average error in my train predictions: " + str((mae_train_lin / y_test.mean()) * 100) + "%")
print("Average error in my test predictions: " + str((mae_test_lin / y_test.mean()) * 100) + "%")
print()
print(f"R2 score for training using Linear Regression = {r2_train}")
print(f"R2 score for testing using Linear Regression = {r2_test}")

MSE for training using Linear Regression = 792545766.5745021
MSE for testing using Linear Regression = 792545766.5745021

MAE for training using Linear Regression = 18619.353071630376
MAE for testing using Linear Regression = 18619.353071630376

Average error in my train predictions: 10.411190271610957%
Average error in my test predictions: 13.27023079580325%

R2 score for training using Linear Regression = 0.867123971445942
R2 score for testing using Linear Regression = 0.6996105013977443


* `Ridge`

In [59]:
ridge_reg = Ridge(alpha=0.0001, max_iter=10000, tol=0.001)
ridge_reg.fit(x_train, y_train)

# Predict
y_pred_train = ridge_reg.predict(x_train)
y_pred_test = ridge_reg.predict(x_test)

# Evaluate
mse_train_lin = mean_squared_error(y_train, y_pred_train)
mse_test_lin = mean_squared_error(y_test, y_pred_test)

mae_train_lin = mean_absolute_error(y_train, y_pred_train)
mae_test_lin = mean_absolute_error(y_test, y_pred_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"MSE for training using Ridge Regression = {mse_train_lin}")
print(f"MSE for testing using Ridge Regression = {mse_train_lin}")
print()
print(f"MAE for training using Ridge Regression = {mae_train_lin}")
print(f"MAE for testing using Ridge Regression = {mae_train_lin}")
print()
print("Average error in my train predictions: " + str((mae_train_lin / y_test.mean()) * 100) + "%")
print("Average error in my test predictions: " + str((mae_test_lin / y_test.mean()) * 100) + "%")
print()
print(f"R2 score for training using Ridge Regression = {r2_train}")
print(f"R2 score for testing using Ridge Regression = {r2_test}")

MSE for training using Ridge Regression = 792545953.5286702
MSE for testing using Ridge Regression = 792545953.5286702

MAE for training using Ridge Regression = 18619.771733552505
MAE for testing using Ridge Regression = 18619.771733552505

Average error in my train predictions: 10.411424370449623%
Average error in my test predictions: 13.268282440356199%

R2 score for training using Ridge Regression = 0.8671239401017239
R2 score for testing using Ridge Regression = 0.6999912959740571


* `Lasso`

In [60]:
lasso_reg = Lasso(alpha=0.001, max_iter=10000, tol=0.001)
lasso_reg.fit(x_train, y_train)

# Predict
y_pred_train = lasso_reg.predict(x_train)
y_pred_test = lasso_reg.predict(x_test)

# Evaluate
mse_train_lin = mean_squared_error(y_train, y_pred_train)
mse_test_lin = mean_squared_error(y_test, y_pred_test)

mae_train_lin = mean_absolute_error(y_train, y_pred_train)
mae_test_lin = mean_absolute_error(y_test, y_pred_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"MSE for training using Lasso Regression = {mse_train_lin}")
print(f"MSE for testing using Lasso Regression = {mse_train_lin}")
print()
print(f"MAE for training using Lasso Regression = {mae_train_lin}")
print(f"MAE for testing using Lasso Regression = {mae_train_lin}")
print()
print("Average error in my train predictions: " + str((mae_train_lin / y_test.mean()) * 100) + "%")
print("Average error in my test predictions: " + str((mae_test_lin / y_test.mean()) * 100) + "%")
print()
print(f"R2 score for training using Lasso Regression = {r2_train}")
print(f"R2 score for testing using Lasso Regression = {r2_test}")

MSE for training using Lasso Regression = 792545766.6992451
MSE for testing using Lasso Regression = 792545766.6992451

MAE for training using Lasso Regression = 18619.36288921909
MAE for testing using Lasso Regression = 18619.36288921909

Average error in my train predictions: 10.41119576121007%
Average error in my test predictions: 13.270188064059948%

R2 score for training using Lasso Regression = 0.867123971425028
R2 score for testing using Lasso Regression = 0.6996155222991638


* `SGD`

In [61]:
sgd_reg = SGDRegressor(penalty="l2", alpha=0.001, eta0=0.001,
                    learning_rate="constant", early_stopping=True, validation_fraction=0.1,
                    n_iter_no_change=5, random_state=42)

sgd_reg.fit(x_train, y_train)

# Predict
y_pred_train_sgd = sgd_reg.predict(x_train)
y_pred_test_sgd = sgd_reg.predict(x_test)

# Evaluate
mse_train_sgd = mean_squared_error(y_train, y_pred_train_sgd)
mse_test_sgd = mean_squared_error(y_test, y_pred_test_sgd)

mae_train_sgd = mean_absolute_error(y_train, y_pred_train)
mae_test_sgd = mean_absolute_error(y_test, y_pred_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f'MSE for Training using SGDRegressor is: {mse_train_sgd:.3f}')
print(f'MSE for Testing using SGDRegressor is: {mse_test_sgd:.3f}')
print()
print(f"MAE for training using SGDRegressor = {mae_train_sgd}")
print(f"MAE for testing using SGDRegressor = {mae_test_sgd}")
print()
print("Average error in my train predictions: " + str((mae_train_sgd / y_test.mean()) * 100) + "%")
print("Average error in my test predictions: " + str((mae_test_sgd / y_test.mean()) * 100) + "%")
print()
print(f"R2 score for training using SGDRegressor = {r2_train}")
print(f"R2 score for testing using SGDRegressor = {r2_test}")

MSE for Training using SGDRegressor is: 9503797173524988334335993053184.000
MSE for Testing using SGDRegressor is: 9243128823205195762716488237056.000

MAE for training using SGDRegressor = 18619.36288921909
MAE for testing using SGDRegressor = 23732.37933854757

Average error in my train predictions: 10.41119576121007%
Average error in my test predictions: 13.270188064059948%

R2 score for training using SGDRegressor = 0.867123971425028
R2 score for testing using SGDRegressor = 0.6996155222991638
